# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [7]:
DROP VIEW IF EXISTS Metricas;
CREATE VIEW Metricas AS
SELECT F.FCID_Code, F.FCID_Desc, F.CGN, F.CG_Subgroup, COUNT(DISTINCT SeqN) Popularidade,
    SUM(Intake) Intake_Sum, AVG(Intake) Intake_AVG, AVG(Intake_BW) Intake_AVG_BW,
    (SELECT COUNT(R.FCID_Code) FROM Recipes R WHERE R.FCID_Code = F.FCID_CODE) AS Recipes
FROM FCID_Description F, Intake I
WHERE F.FCID_Code = I.FCID_Code
GROUP BY I.FCID_Code;

In [8]:
SELECT * FROM Metricas;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

2)-Nessa query, fizemos uma correlação entre as métricas Recipes (número de receitas nas quais aquele produto participa) e Popularidade (número de pessoas que consumiram aquele produto). Para isso, criamos intervalos(Δ50) de popularidades e calculamos a quantidade média de receitas para cada um desses intervalos. Assim é possível enxergar que o número de receitas tende a crescer, junto com os valores de popularidade.

In [9]:
SELECT FLOOR(Popularidade / 50) * 50 AS IntervaloPopularidade, AVG(Recipes)
FROM Metricas M
GROUP BY FLOOR(Popularidade / 50) * 50
ORDER BY IntervaloPopularidade;

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

Fizemos uma classificação de faixa etária, com base no peso de cada consumidor.

In [20]:
DROP VIEW IF EXISTS Consumidor;
CREATE VIEW Consumidor AS
SELECT DISTINCT SeqN, CAST(Intake / Intake_BW AS INTEGER) as Peso, 
    CASE
        WHEN CAST(Intake / Intake_BW AS INTEGER) <= 5 THEN 'newborn'
        WHEN CAST(Intake / Intake_BW AS INTEGER) <= 35 THEN 'child'
        ELSE 'adult'
    END AS Faixa_Etaria FROM Intake;

In [21]:
SELECT * FROM Consumidor;

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [13]:
DROP VIEW IF EXISTS Metricas_Faixa_Etaria;
CREATE VIEW Metricas_Faixa_Etaria AS
SELECT FCID_Code, FCID_Desc, Intake_AVG / Intake_AVG_BW as Peso_Medio,
    CASE
        WHEN Intake_AVG / Intake_AVG_BW <= 5 THEN 'newborn'
        WHEN Intake_AVG / Intake_AVG_BW <= 35 THEN 'child'
        ELSE 'adult'
    END AS Faixa_Etaria_Consumidor_Medio
FROM Metricas;

In [22]:
SELECT * FROM Metricas_Faixa_Etaria;